In [1]:
from pprint import pprint

In [2]:
import fitz  # import package PyMuPDF

# Open some document, for example a PDF (could also be EPUB, XPS, etc.)
doc = fitz.open("Reports/tcs.pdf")

In [3]:
def is_rectangle_outside_all(input_rectangle, all_rectangles):
    x0_input, y0_input, x1_input, y1_input = input_rectangle

    for rectangle in all_rectangles:
        x0, y0, x1, y1 = rectangle

        # Check if input rectangle is entirely to the left or right of the current rectangle
        if x1_input <= x0 or x0_input >= x1:
            continue

        # Check if input rectangle is entirely above or below the current rectangle
        if y1_input <= y0 or y0_input >= y1:
            continue

        # If any overlap is found, return False
        return False

    # If no overlap is found with any rectangle, return True
    return True
def is_rectangle_outside_all(input_rectangle, all_rectangles):
    x0_input, y0_input, x1_input, y1_input = input_rectangle

    for rectangle in all_rectangles:
        x0, y0, x1, y1 = rectangle

        # Check if input rectangle is entirely to the left or right of the current rectangle
        if x1_input <= x0 or x0_input >= x1:
            continue

        # Check if input rectangle is entirely above or below the current rectangle
        if y1_input <= y0 or y0_input >= y1:
            continue

        # If any overlap is found, return False
        return False

    # If no overlap is found with any rectangle, return True
    return True


In [4]:
page = doc[106]  # this is the first page

In [5]:
blocks = page.get_text("dict")["blocks"]

In [6]:
tabs = page.find_tables()
l=len(tabs.tables)
page_all_table_bbox=[]
for i in range(l):
    page_all_table_bbox.append(tabs[i].bbox)

In [7]:
tabs[0].extract()

[['', 'FY 2023', 'FY 2022', 'FY 2023', 'FY 2022', 'FY 2023', 'FY 2022'],
 [None, 'Current', None, 'Non-current', None, 'Total funds invested', None],
 ['Investments in mutual funds, Government\nsecurities and others',
  '36,897',
  '30,262',
  '230',
  '187',
  '37,127',
  '30,449'],
 ['Deposits with banks',
  '8,223',
  '15,784',
  '1,334',
  '1,232',
  '9,557',
  '17,016'],
 ['Inter-corporate deposits', '846', '6,074', '170', '303', '1,016', '6,377'],
 ['Cash and bank balances', '2,124', '2,211', '-', '-', '2,124', '2,211'],
 ['Total', '48,090', '54,331', '1,734', '1,722', '49,824', '56,053']]

In [8]:
tabs[0].to_pandas()

,Col0,1-FY 2023,2-FY 2022,3-FY 2023,4-FY 2022,5-FY 2023,6-FY 2022
0,None,Current,None,Non-current,None,Total funds invested,None
1,"Investments in mutual funds, Government\nsecur...","36,897","30,262",230,187,"37,127","30,449"
2,Deposits with banks,"8,223","15,784","1,334","1,232","9,557","17,016"
3,Inter-corporate deposits,846,"6,074",170,303,"1,016","6,377"
4,Cash and bank balances,"2,124","2,211",-,-,"2,124","2,211"
5,Total,"48,090","54,331","1,734","1,722","49,824","56,053"


In [9]:
page_all_table_bbox

[(54.125, 102.65790608723958, 540.4500045776367, 202.90790608723958),
 (54.125, 258.5657846551192, 540.4500045776367, 649.0657276826746)]

In [10]:
text=""
table_counter=0
for b in blocks:  # iterate through the text blocks
   if "lines" in b: 
    for l in b["lines"]:  # iterate through the text lines
        for s in l["spans"]:  # iterate through the text spans
            if is_rectangle_outside_all(b["bbox"], page_all_table_bbox):
                text += ' ' + s["text"]
            else:
                text+= ' ' + f'{tabs[table_counter].extract()}'
                table_counter=1

In [11]:
page.get_textbox(rect=(54.125, 192.6009286733774, 539.875, 325.8509063720703))

'Total\n48,090\n54,331\n1,734\n1,722\n49,824\n56,053\nTotal invested funds include ₹2,080 crore and ₹1,722 crore for FY 2023 and FY 2022, respectively, pertaining to trusts and \nTCS Foundation held for specified purposes.\nRatio Analysis – 10 years \nInd As\nIndian GAAP\nUnit\nFY \n2023\n FY \n2022\n FY \n2021*\n FY \n2021\n FY \n2020\n FY \n2019\n FY \n2018\n FY \n2017\n FY \n2016\n FY \n2015#\n FY \n2015\n FY \n2014\nRatios - Financial Performance\nEmployee Cost / Total Revenue\n%\n 56.6\n 56.1\n 55.9\n 55.9\n 54.8\n 53.4\n 53.9\n 52.2\n 50.9\n 51.0\n 53.8\n 49.5\nOther Operating Cost / Total '

In [12]:
print(text)

 Management Discussion and Analysis   |  107 Integrated Annual Report 2022-23 Overview of Funds Invested  Funds invested exclude earmarked balances with banks and equity shares measured at fair value through other comprehensive  income. ( `  Crore) [['', 'FY 2023', 'FY 2022', 'FY 2023', 'FY 2022', 'FY 2023', 'FY 2022'], [None, 'Current', None, 'Non-current', None, 'Total funds invested', None], ['Investments in mutual funds, Government\nsecurities and others', '36,897', '30,262', '230', '187', '37,127', '30,449'], ['Deposits with banks', '8,223', '15,784', '1,334', '1,232', '9,557', '17,016'], ['Inter-corporate deposits', '846', '6,074', '170', '303', '1,016', '6,377'], ['Cash and bank balances', '2,124', '2,211', '-', '-', '2,124', '2,211'], ['Total', '48,090', '54,331', '1,734', '1,722', '49,824', '56,053']] [['', '', 'Ind As', None, None, None, None, None, None, None, None, 'Indian GAAP', None, None], [None, 'Unit', 'FY\n2023', 'FY\n2022', 'FY\n2021*', 'FY\n2021', 'FY\n2020', 'FY\n2

In [13]:
page.get_textbox(rect=(54.125, 192.6009286733774, 539.875, 325.8509063720703))

'Total\n48,090\n54,331\n1,734\n1,722\n49,824\n56,053\nTotal invested funds include ₹2,080 crore and ₹1,722 crore for FY 2023 and FY 2022, respectively, pertaining to trusts and \nTCS Foundation held for specified purposes.\nRatio Analysis – 10 years \nInd As\nIndian GAAP\nUnit\nFY \n2023\n FY \n2022\n FY \n2021*\n FY \n2021\n FY \n2020\n FY \n2019\n FY \n2018\n FY \n2017\n FY \n2016\n FY \n2015#\n FY \n2015\n FY \n2014\nRatios - Financial Performance\nEmployee Cost / Total Revenue\n%\n 56.6\n 56.1\n 55.9\n 55.9\n 54.8\n 53.4\n 53.9\n 52.2\n 50.9\n 51.0\n 53.8\n 49.5\nOther Operating Cost / Total '

In [14]:
print(page.get_text())

Management Discussion and Analysis   |  107
Integrated Annual Report 2022-23
Overview of Funds Invested 
Funds invested exclude earmarked balances with banks and equity shares measured at fair value through other comprehensive 
income.
(` Crore)
FY 2023
FY 2022
FY 2023
FY 2022
FY 2023
FY 2022
Current
Non-current
Total funds invested
Investments in mutual funds, Government 
securities and others
36,897
30,262
230
187
37,127
30,449
Deposits with banks
8,223
15,784
1,334
1,232
9,557
17,016
Inter-corporate deposits
846
6,074
170
303
1,016
6,377
Cash and bank balances
2,124
2,211
-
-
2,124
2,211
Total
48,090
54,331
1,734
1,722
49,824
56,053
Total invested funds include ₹2,080 crore and ₹1,722 crore for FY 2023 and FY 2022, respectively, pertaining to trusts and 
TCS Foundation held for specified purposes.
Ratio Analysis – 10 years 
Ind As
Indian GAAP
Unit
FY 
2023
 FY 
2022
 FY 
2021*
 FY 
2021
 FY 
2020
 FY 
2019
 FY 
2018
 FY 
2017
 FY 
2016
 FY 
2015#
 FY 
2015
 FY 
2014
Ratios - Financi

In [15]:
tabs = page.find_tables()
print(f"{len(tabs.tables)} table(s) on {page}")
# pprint(tabs[0].extract())

2 table(s) on page 106 of Reports/tcs.pdf


In [16]:
tabs[0].extract()

[['', 'FY 2023', 'FY 2022', 'FY 2023', 'FY 2022', 'FY 2023', 'FY 2022'],
 [None, 'Current', None, 'Non-current', None, 'Total funds invested', None],
 ['Investments in mutual funds, Government\nsecurities and others',
  '36,897',
  '30,262',
  '230',
  '187',
  '37,127',
  '30,449'],
 ['Deposits with banks',
  '8,223',
  '15,784',
  '1,334',
  '1,232',
  '9,557',
  '17,016'],
 ['Inter-corporate deposits', '846', '6,074', '170', '303', '1,016', '6,377'],
 ['Cash and bank balances', '2,124', '2,211', '-', '-', '2,124', '2,211'],
 ['Total', '48,090', '54,331', '1,734', '1,722', '49,824', '56,053']]

In [20]:
mudf = tabs[0].to_pandas()

In [21]:
mudf

,Col0,1-FY 2023,2-FY 2022,3-FY 2023,4-FY 2022,5-FY 2023,6-FY 2022
0,None,Current,None,Non-current,None,Total funds invested,None
1,"Investments in mutual funds, Government\nsecur...","36,897","30,262",230,187,"37,127","30,449"
2,Deposits with banks,"8,223","15,784","1,334","1,232","9,557","17,016"
3,Inter-corporate deposits,846,"6,074",170,303,"1,016","6,377"
4,Cash and bank balances,"2,124","2,211",-,-,"2,124","2,211"
5,Total,"48,090","54,331","1,734","1,722","49,824","56,053"


In [22]:
import camelot

In [37]:
page_table = camelot.read_pdf('Reports/tcs.pdf', flavor='stream', pages="72")

In [38]:
page_table

<TableList n=3>

In [51]:
dir(page_table[1])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bbox',
 '_image',
 '_segments',
 '_text',
 '_textedges',
 'accuracy',
 'cells',
 'cols',
 'data',
 'df',
 'flavor',
 'order',
 'page',
 'parsing_report',
 'rows',
 'set_all_edges',
 'set_border',
 'set_edges',
 'set_span',
 'shape',
 'to_csv',
 'to_excel',
 'to_html',
 'to_json',
 'to_sqlite',
 'whitespace']

In [63]:
df=page_table[1].to_json("test.json")

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__func__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']